In [1]:
import pandas as pd
import json
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from collections import defaultdict

In [2]:
# generate state action pair from its children who has the highest value
def load_data(filename, sep='\t', visit_threshold=5, col='Game_Features'):
    df = pd.read_csv(filename, sep=sep)
    features = json.loads(df.iloc[0][col]).keys()
    data = {"Action": []}

    for feature in features:
        data[feature] = []

    for _, row in df.iterrows():
        state_features = json.loads(row[col])
        children = df[df['Parent_Name'] == row['Name']]
        children = children[children['Visits'] >= visit_threshold]
        if children.shape[0]:
            child = children.nlargest(1, 'Value')
            child_action = child.iloc[0]['Action_Name']
            for feature in state_features:
                data[feature].append(state_features[feature])
            data['Action'].append(child_action)
            
    return pd.DataFrame(data)

## Decision Tree based on game state (low-level features)

In [3]:
# dfs = []

# for i in os.listdir("DotsAndBoxes"):
#     if i.split(".")[-1] != 'csv':
#         continue
#     print(i)
#     dfs.append(load_data(f"DotsAndBoxes/{i}", visit_threshold=1, col='Game_State'))

# df = pd.concat(dfs)
# df.to_csv('DB_game_states.csv')
# df

df = pd.read_csv("DB_game_states.csv", index_col='Unnamed: 0')
df

,Action,Edge_Owner_6061,Edge_Owner_6263,Edge_Owner_6465,Edge_Owner_5060,Edge_Owner_5262,Edge_Owner_5464,Edge_Owner_6171,Edge_Owner_6373,Edge_Owner_6575,...,Cell_Owner_63,Cell_Edge_Count_63,Cell_Owner_20,Cell_Edge_Count_20,Cell_Owner_64,Cell_Edge_Count_64,Cell_Owner_21,Cell_Edge_Count_21,Cell_Owner_22,Cell_Edge_Count_22
0,"(0,3) -> (0,4)",-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,0,-1,0,-1,0,-1,0,-1,0
0,"(0,1) -> (0,2)",-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,1,-1,1,-1,1,-1,1
1,"(0,3) -> (0,4)",-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,1,-1,1,-1,1,-1,1
2,"(1,2) -> (1,3)",-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,1,-1,1,-1,2,-1,2
3,"(4,1) -> (4,2)",-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,1,-1,1,-1,1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,"(2,0) -> (2,1)",1,-1,1,1,0,1,0,0,-1,...,-1,3,-1,1,-1,1,-1,2,1,4
40,"(1,0) -> (1,1)",1,-1,1,1,0,1,0,0,-1,...,-1,3,-1,1,-1,1,-1,2,1,4
41,"(3,1) -> (3,2)",1,-1,1,1,0,1,-1,0,-1,...,1,4,-1,1,-1,2,-1,2,1,4
42,"(6,5) -> (7,5)",1,-1,1,1,0,1,-1,0,-1,...,1,4,-1,1,-1,2,-1,2,1,4


In [4]:
X = df.drop(columns=['Action'])
y = df['Action']

# Max leaf is the number of action
max_leaf = len(df['Action'].unique())
features = X.columns

In [5]:
clf = DecisionTreeClassifier(max_leaf_nodes=max_leaf, random_state=42)
clf.fit(X, y)

DecisionTreeClassifier(max_leaf_nodes=82, random_state=42)

In [6]:
text_representation = tree.export_text(clf, feature_names=list(features))
print(text_representation)

|--- Cell_Edge_Count_02 <= 3.50
|   |--- Cell_Owner_21 <= -0.50
|   |   |--- Cell_Owner_63 <= -0.50
|   |   |   |--- Edge_Owner_0102 <= -0.50
|   |   |   |   |--- Cell_Edge_Count_03 <= 3.50
|   |   |   |   |   |--- class: (0,1) -> (0,2)
|   |   |   |   |--- Cell_Edge_Count_03 >  3.50
|   |   |   |   |   |--- class: (0,1) -> (0,2)
|   |   |   |--- Edge_Owner_0102 >  -0.50
|   |   |   |   |--- Cell_Owner_12 <= 0.50
|   |   |   |   |   |--- Cell_Owner_31 <= 0.50
|   |   |   |   |   |   |--- Edge_Owner_0212 <= -0.50
|   |   |   |   |   |   |   |--- class: (0,2) -> (1,2)
|   |   |   |   |   |   |--- Edge_Owner_0212 >  -0.50
|   |   |   |   |   |   |   |--- class: (0,0) -> (1,0)
|   |   |   |   |   |--- Cell_Owner_31 >  0.50
|   |   |   |   |   |   |--- class: (0,0) -> (0,1)
|   |   |   |   |--- Cell_Owner_12 >  0.50
|   |   |   |   |   |--- class: (1,4) -> (1,5)
|   |   |--- Cell_Owner_63 >  -0.50
|   |   |   |--- Cell_Edge_Count_14 <= 3.50
|   |   |   |   |--- Edge_Owner_2232 <= -0.50
|   

In [29]:
action_decisions = []
current_action_set = []

n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold

def retrieve_tree(tree, classes, node_id):
    is_split_node = tree.children_left[node_id] != tree.children_right[node_id]
    
    current_action_set.append(features[tree.feature[node_id]])
    
    if is_split_node:
        retrieve_tree(tree, classes, children_left[node_id])
        retrieve_tree(tree, classes, children_right[node_id])
    else:
        action_decisions.append(current_action_set + [classes[np.array(tree.value[node_id][0]).argmax()]])
        
    current_action_set.pop()

retrieve_tree(clf.tree_, clf.classes_, 0)

In [8]:
action_feature_dict = defaultdict(list)

for action_decision in action_decisions:
    action_feature_dict[action_decision[-1]].append(action_decision[: -1])
    
for action_name, features in action_feature_dict.items():
    feature_set = set()
    for feature_list in features:
        for feature in feature_list:
            feature_set.add(feature)
        
    print(f"Action Name: {action_name}")
    print(f"Feature List: [{', '.join(list(feature_set))}]")
    print(f"Paths({len(features)}):")
    for idx, feature_list in enumerate(features, 1):
        print(f"\tPath {idx}: {', '.join(feature_list)}")
    print("-" * 115)
    
print("The excluded action list:\n\t", end='')
print('\n\t'.join(set(df['Action'].unique()) ^ set(action_feature_dict.keys())))

Action Name: (0,1) -> (0,2)
Feature List: [Edge_Owner_2232, Edge_Owner_0102, Cell_Edge_Count_14, Cell_Edge_Count_02, Cell_Edge_Count_03, Cell_Owner_21, Cell_Owner_22, Cell_Owner_63]
Paths(3):
	Path 1: Cell_Edge_Count_02, Cell_Owner_21, Cell_Owner_63, Edge_Owner_0102, Cell_Edge_Count_03, Cell_Owner_22
	Path 2: Cell_Edge_Count_02, Cell_Owner_21, Cell_Owner_63, Edge_Owner_0102, Cell_Edge_Count_03, Cell_Owner_22
	Path 3: Cell_Edge_Count_02, Cell_Owner_21, Cell_Owner_63, Cell_Edge_Count_14, Edge_Owner_2232, Cell_Owner_22
-------------------------------------------------------------------------------------------------------------------
Action Name: (0,2) -> (1,2)
Feature List: [Edge_Owner_0102, Cell_Owner_03, Edge_Owner_1314, Edge_Owner_1213, Cell_Owner_12, Cell_Edge_Count_02, Cell_Owner_22, Edge_Owner_1222, Edge_Owner_7273, Edge_Owner_3334, Cell_Owner_31, Edge_Owner_4445, Cell_Owner_21, Edge_Owner_0212, Cell_Owner_63]
Paths(5):
	Path 1: Cell_Edge_Count_02, Cell_Owner_21, Cell_Owner_63, Edge

## Decision Tree based on game features (high-level features)

In [3]:
# dfs = []

# for i in os.listdir("DotsAndBoxes"):
#     if i.split(".")[-1] != 'csv':
#         continue
#     print(i)
#     dfs.append(load_data(f"DotsAndBoxes/{i}", visit_threshold=1))

# df = pd.concat(dfs)
# df.to_csv("db_game_features.csv")

df = pd.read_csv("db_game_features.csv", index_col='Unnamed: 0')
df

,Action,SCORE,SCORE_ADV,ORDINAL,OUR_TURN,HAS_WON,FINAL_ORD,ROUND,NO_BOXES,ONE_BOXES,TWO_BOXES,THREE_BOXES,OPPONENTS_FILLED_BOXES,OWNED_FILLED_BOXES
0,"(5,0) -> (6,0)",1.10,2.2,0.5,1.0,0.0,0.0,0.0,2.0,3.0,7.0,1.0,0.0,22.0
1,"(5,5) -> (6,5)",1.15,2.3,0.5,1.0,0.0,0.0,0.0,2.0,3.0,7.0,0.0,0.0,23.0
2,"(5,0) -> (6,0)",1.10,2.2,0.5,1.0,0.0,0.0,0.0,2.0,2.0,6.0,3.0,0.0,22.0
3,"(5,0) -> (6,0)",1.10,2.1,0.5,1.0,0.0,0.0,0.0,1.0,3.0,6.0,2.0,1.0,22.0
4,"(5,0) -> (6,0)",1.10,2.1,0.5,1.0,0.0,0.0,0.0,1.0,3.0,7.0,1.0,1.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88236,"(2,0) -> (3,0)",1.15,2.1,0.5,1.0,0.0,0.0,0.0,1.0,3.0,2.0,4.0,2.0,23.0
88237,"(2,0) -> (2,1)",1.15,2.2,0.5,1.0,0.0,0.0,0.0,1.0,1.0,5.0,4.0,1.0,23.0
88238,"(3,3) -> (3,4)",1.15,2.2,0.5,1.0,0.0,0.0,0.0,1.0,2.0,3.0,5.0,1.0,23.0
88239,"(1,0) -> (1,1)",1.15,2.1,0.5,1.0,0.0,0.0,0.0,0.0,3.0,5.0,2.0,2.0,23.0


In [4]:
X = df.drop(columns=['Action'])
y = df['Action']

# Max leaf is the number of action
max_leaf = len(df['Action'].unique())
features = X.columns

In [11]:
clf = DecisionTreeClassifier(max_leaf_nodes=max_leaf, random_state=42)
clf.fit(X, y)

DecisionTreeClassifier(max_leaf_nodes=82, random_state=42)

In [12]:
text_representation = tree.export_text(clf, feature_names=list(features))
print(text_representation)

|--- SCORE_ADV <= 0.65
|   |--- NO_BOXES <= 5.50
|   |   |--- NO_BOXES <= 2.50
|   |   |   |--- ONE_BOXES <= 17.50
|   |   |   |   |--- class: (0,0) -> (0,1)
|   |   |   |--- ONE_BOXES >  17.50
|   |   |   |   |--- class: (0,0) -> (1,0)
|   |   |--- NO_BOXES >  2.50
|   |   |   |--- SCORE <= 0.28
|   |   |   |   |--- class: (0,3) -> (0,4)
|   |   |   |--- SCORE >  0.28
|   |   |   |   |--- class: (0,3) -> (0,4)
|   |--- NO_BOXES >  5.50
|   |   |--- OPPONENTS_FILLED_BOXES <= 0.50
|   |   |   |--- NO_BOXES <= 33.50
|   |   |   |   |--- TWO_BOXES <= 9.50
|   |   |   |   |   |--- class: (0,3) -> (0,4)
|   |   |   |   |--- TWO_BOXES >  9.50
|   |   |   |   |   |--- class: (0,1) -> (0,2)
|   |   |   |--- NO_BOXES >  33.50
|   |   |   |   |--- class: (0,1) -> (0,2)
|   |   |--- OPPONENTS_FILLED_BOXES >  0.50
|   |   |   |--- class: (0,3) -> (0,4)
|--- SCORE_ADV >  0.65
|   |--- OWNED_FILLED_BOXES <= 17.50
|   |   |--- OPPONENTS_FILLED_BOXES <= 0.50
|   |   |   |--- TWO_BOXES <= 9.50
|   |   

In [13]:
action_decisions = []
current_action_set = []

n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold

retrieve_tree(clf.tree_, clf.classes_, 0)

In [14]:
action_feature_dict = defaultdict(list)

for action_decision in action_decisions:
    action_feature_dict[action_decision[-1]].append(action_decision[: -1])
    
for action_name, features in action_feature_dict.items():
    feature_set = set()
    for feature_list in features:
        for feature in feature_list:
            feature_set.add(feature)
        
    print(f"Action Name: {action_name}")
    print(f"Feature List: [{', '.join(list(feature_set))}]")
    print(f"Paths({len(features)}):")
    for idx, feature_list in enumerate(features, 1):
        print(f"\tPath {idx}: {', '.join(feature_list)}")
    print("-" * 115)
    
print("The excluded action list:\n\t", end='')
print('\n\t'.join(set(df['Action'].unique()) ^ set(action_feature_dict.keys())))

Action Name: (0,0) -> (0,1)
Feature List: [ONE_BOXES, SCORE_ADV, SCORE, TWO_BOXES, THREE_BOXES, OPPONENTS_FILLED_BOXES, OWNED_FILLED_BOXES, FINAL_ORD, NO_BOXES]
Paths(22):
	Path 1: SCORE_ADV, NO_BOXES, NO_BOXES, ONE_BOXES, OPPONENTS_FILLED_BOXES
	Path 2: SCORE_ADV, OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES, TWO_BOXES, ONE_BOXES, NO_BOXES, NO_BOXES, OPPONENTS_FILLED_BOXES
	Path 3: SCORE_ADV, OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES, TWO_BOXES, ONE_BOXES, NO_BOXES, NO_BOXES, OPPONENTS_FILLED_BOXES
	Path 4: SCORE_ADV, OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES, TWO_BOXES, ONE_BOXES, NO_BOXES, NO_BOXES, OPPONENTS_FILLED_BOXES
	Path 5: SCORE_ADV, OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES, TWO_BOXES, ONE_BOXES, NO_BOXES, OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES
	Path 6: SCORE_ADV, OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES, TWO_BOXES, NO_BOXES, OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES
	Path 7: SCORE_ADV, OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES, TWO_BOXES, NO_BOXES, OWNED_FILLED_BO

In [35]:
df_seperated = df[df['Action'] == df['Action'].unique()[0]]
df_seperated_not = df[df['Action'] != df['Action'].unique()[0]].sample(df_seperated.shape[0])
df_seperated_not['Action'] = 'Other'
df_test = pd.concat([df_seperated, df_seperated_not])

In [55]:
X = df_test.drop(columns=['Action'])
y = df_test['Action']

# Max leaf is the number of action
max_leaf = len(df_test['Action'].unique())
features = X.columns

In [56]:
clf = DecisionTreeClassifier(random_state=42, max_leaf_nodes=6)
clf.fit(X, y)

DecisionTreeClassifier(max_leaf_nodes=6, random_state=42)

In [57]:
text_representation = tree.export_text(clf, feature_names=list(features))
print(text_representation)

|--- OWNED_FILLED_BOXES <= 10.50
|   |--- class: Other
|--- OWNED_FILLED_BOXES >  10.50
|   |--- OWNED_FILLED_BOXES <= 17.50
|   |   |--- OPPONENTS_FILLED_BOXES <= 0.50
|   |   |   |--- class: Other
|   |   |--- OPPONENTS_FILLED_BOXES >  0.50
|   |   |   |--- class: (5,0) -> (6,0)
|   |--- OWNED_FILLED_BOXES >  17.50
|   |   |--- THREE_BOXES <= 7.50
|   |   |   |--- NO_BOXES <= 0.50
|   |   |   |   |--- class: (5,0) -> (6,0)
|   |   |   |--- NO_BOXES >  0.50
|   |   |   |   |--- class: (5,0) -> (6,0)
|   |   |--- THREE_BOXES >  7.50
|   |   |   |--- class: Other



In [58]:
action_decisions = []
current_action_set = []

n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold

retrieve_tree(clf.tree_, clf.classes_, 0)

In [59]:
action_feature_dict = defaultdict(list)

for action_decision in action_decisions:
    action_feature_dict[action_decision[-1]].append(action_decision[: -1])
    
for action_name, features in action_feature_dict.items():
    feature_set = set()
    for feature_list in features:
        for feature in feature_list:
            feature_set.add(feature)
        
    print(f"Action Name: {action_name}")
    print(f"Feature List: [{', '.join(list(feature_set))}]")
    print(f"Paths({len(features)}):")
    for idx, feature_list in enumerate(features, 1):
        print(f"\tPath {idx}: {', '.join(feature_list)}")
    print("-" * 115)
    
print("The excluded action list:\n\t", end='')

Action Name: Other
Feature List: [OWNED_FILLED_BOXES, THREE_BOXES, OPPONENTS_FILLED_BOXES]
Paths(3):
	Path 1: OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES
	Path 2: OWNED_FILLED_BOXES, OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES, OPPONENTS_FILLED_BOXES
	Path 3: OWNED_FILLED_BOXES, OWNED_FILLED_BOXES, THREE_BOXES, OPPONENTS_FILLED_BOXES
-------------------------------------------------------------------------------------------------------------------
Action Name: (5,0) -> (6,0)
Feature List: [OWNED_FILLED_BOXES, NO_BOXES, THREE_BOXES, OPPONENTS_FILLED_BOXES]
Paths(3):
	Path 1: OWNED_FILLED_BOXES, OWNED_FILLED_BOXES, OPPONENTS_FILLED_BOXES, OPPONENTS_FILLED_BOXES
	Path 2: OWNED_FILLED_BOXES, OWNED_FILLED_BOXES, THREE_BOXES, NO_BOXES, OPPONENTS_FILLED_BOXES
	Path 3: OWNED_FILLED_BOXES, OWNED_FILLED_BOXES, THREE_BOXES, NO_BOXES, OPPONENTS_FILLED_BOXES
-------------------------------------------------------------------------------------------------------------------
The excluded action list:
	

## Logistic Regression based on game state

In [15]:
df = pd.read_csv("DB_game_states.csv", index_col='Unnamed: 0')
df = df[df.columns.drop(list(df.filter(regex='Cell_Owner_')))]
df

,Action,Edge_Owner_6061,Edge_Owner_6263,Edge_Owner_6465,Edge_Owner_5060,Edge_Owner_5262,Edge_Owner_5464,Edge_Owner_6171,Edge_Owner_6373,Edge_Owner_6575,...,Cell_Edge_Count_33,Cell_Edge_Count_34,Cell_Edge_Count_60,Cell_Edge_Count_61,Cell_Edge_Count_62,Cell_Edge_Count_63,Cell_Edge_Count_20,Cell_Edge_Count_64,Cell_Edge_Count_21,Cell_Edge_Count_22
0,"(0,3) -> (0,4)",-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
0,"(0,1) -> (0,2)",-1,-1,-1,-1,-1,-1,-1,-1,-1,...,2,0,0,0,0,1,1,1,1,1
1,"(0,3) -> (0,4)",-1,-1,-1,-1,-1,-1,-1,-1,-1,...,2,0,0,0,0,1,1,1,1,1
2,"(1,2) -> (1,3)",-1,-1,-1,-1,-1,-1,-1,-1,-1,...,2,0,0,0,0,1,1,1,2,2
3,"(4,1) -> (4,2)",-1,-1,-1,-1,-1,-1,-1,-1,-1,...,2,0,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,"(2,0) -> (2,1)",1,-1,1,1,0,1,0,0,-1,...,0,1,3,3,1,3,1,1,2,4
40,"(1,0) -> (1,1)",1,-1,1,1,0,1,0,0,-1,...,0,1,3,3,1,3,1,1,2,4
41,"(3,1) -> (3,2)",1,-1,1,1,0,1,-1,0,-1,...,0,1,2,2,1,4,1,2,2,4
42,"(6,5) -> (7,5)",1,-1,1,1,0,1,-1,0,-1,...,0,2,3,2,1,4,1,2,2,4


In [16]:
X = df.drop(columns=['Action'])
y = df['Action']

In [17]:
model = LogisticRegression(random_state=42, max_iter=500)
model.fit(X, y)

C:\Users\roven\anaconda3\envs\phd\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=500, random_state=42)

In [18]:
model.score(X, y)

0.11664645686245623

In [19]:
def weights_importance(model, action_idx, top=10, precision=5):
    def float_precision(num):
        return f'%.{precision}f' % num
    
    
    if action_idx >= len(model.classes_):
        print("The action idx is out of range")
        return
    
    print("Action Name:", model.classes_[action_idx])
    weights = model.coef_[action_idx]
    idxs = weights.argsort()[::-1][: top]
    print(f"Top {top} features: ", ', '.join(model.feature_names_in_[idxs]))
    print(f"Top {top} features importance: ", ', '.join(map(float_precision, weights[idxs])))

In [20]:
weights_importance(model, 0)

Action Name: (0,0) -> (0,1)
Top 10 features:  Cell_Edge_Count_43, Cell_Edge_Count_52, Edge_Owner_3343, Edge_Owner_4243, Cell_Edge_Count_54, Cell_Edge_Count_34, Cell_Edge_Count_30, Edge_Owner_4555, Edge_Owner_3242, Cell_Edge_Count_23
Top 10 features importance:  0.80203, 0.71530, 0.65263, 0.57732, 0.55508, 0.52974, 0.47907, 0.47568, 0.40712, 0.40051


In [21]:
weights_importance(model, 10)

Action Name: (0,5) -> (1,5)
Top 10 features:  Cell_Edge_Count_14, Edge_Owner_0414, Cell_Edge_Count_44, Cell_Edge_Count_52, Edge_Owner_1222, Cell_Edge_Count_61, Edge_Owner_1121, Cell_Edge_Count_63, Edge_Owner_7273, Edge_Owner_2535
Top 10 features importance:  0.62924, 0.62890, 0.60147, 0.49253, 0.49165, 0.49028, 0.48840, 0.48541, 0.40970, 0.39365


In [22]:
weights_importance(model, 50)

Action Name: (4,3) -> (4,4)
Top 10 features:  Edge_Owner_3132, Cell_Edge_Count_53, Edge_Owner_3141, Edge_Owner_6465, Cell_Edge_Count_50, Edge_Owner_4142, Edge_Owner_7172, Cell_Edge_Count_11, Cell_Edge_Count_24, Edge_Owner_5565
Top 10 features importance:  0.65480, 0.36659, 0.36456, 0.35971, 0.35873, 0.35475, 0.34190, 0.33186, 0.32904, 0.32659


## Logistic Regression based on game state difference (Use all of state_action_state pair)

In [23]:
class State:
    def __init__(self, features_dict):
        self.features_dict = features_dict
        
    def distance(self, state):
        distance = 0
        for key, feature in self.feautres_dict.items():
            distance += (state.features_dict[key] - feature) ** 2
            
        return math.sqrt(distance)
    
    def feature_difference(self, state):
        difference_dict = {}
        for key, feature in self.features_dict.items():
            difference_dict[key] = feature - state.features_dict[key]
            
        return difference_dict
        
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.features_dict == other.features_dict
        else:
            return False

In [24]:
transition_df_raw = pd.read_csv("db_game_state_transition.csv")

transition_df_dict = {'action': []}

for feature in json.loads(transition_df_raw.iloc[0]['state_0']).keys():
    transition_df_dict[feature] = []

for _, row in transition_df_raw.iterrows():
    state_0 = State(json.loads(row['state_0']))
    action = row['action']
    state_1 = State(json.loads(row['state_1']))
    difference_dict = state_1.feature_difference(state_0)
    
    transition_df_dict['action'].append(action)
    
    for key, value in difference_dict.items():
        transition_df_dict[key].append(value)
        
transition_df = pd.DataFrame.from_dict(transition_df_dict)
transition_df.head()

,action,Edge_Owner_6061,Edge_Owner_6263,Edge_Owner_6465,Edge_Owner_5060,Edge_Owner_5262,Edge_Owner_5464,Edge_Owner_6171,Edge_Owner_6373,Edge_Owner_6575,...,Cell_Owner_63,Cell_Edge_Count_63,Cell_Owner_20,Cell_Edge_Count_20,Cell_Owner_64,Cell_Edge_Count_64,Cell_Owner_21,Cell_Edge_Count_21,Cell_Owner_22,Cell_Edge_Count_22
0,"(0,3) -> (0,4)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(1,0) -> (1,1)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(2,1) -> (2,2)",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,2,0,0
3,"(3,3) -> (4,3)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(3,5) -> (4,5)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
X = transition_df.drop(columns=['action'])
y = transition_df['action']

In [26]:
model = LogisticRegression(random_state=42, max_iter=500)
model.fit(X, y)

LogisticRegression(max_iter=500, random_state=42)

In [27]:
model.score(X, y)

0.4848096552556201

In [28]:
weights_importance(model, 0)

Action Name: (0,0) -> (0,1)
Top 10 features:  Cell_Edge_Count_00, Edge_Owner_0102, Cell_Edge_Count_11, Edge_Owner_0212, Edge_Owner_0001, Edge_Owner_6373, Edge_Owner_3040, Edge_Owner_1323, Edge_Owner_4041, Edge_Owner_1020
Top 10 features importance:  3.69287, 1.20471, 1.15071, 1.08928, 0.99442, 0.85016, 0.84841, 0.82021, 0.77724, 0.76942


In [29]:
weights_importance(model, 10)

Action Name: (0,5) -> (1,5)
Top 10 features:  Cell_Edge_Count_04, Edge_Owner_3334, Edge_Owner_5051, Edge_Owner_4041, Edge_Owner_1525, Edge_Owner_1222, Edge_Owner_6061, Edge_Owner_4151, Edge_Owner_2425, Edge_Owner_2434
Top 10 features importance:  4.99043, 1.59577, 1.49116, 1.41046, 1.39765, 1.33727, 1.32591, 1.29522, 1.29174, 1.26139


In [30]:
weights_importance(model, 50)

Action Name: (4,3) -> (4,4)
Top 10 features:  Edge_Owner_4344, Cell_Edge_Count_43, Edge_Owner_4445, Cell_Edge_Count_33, Edge_Owner_4243, Edge_Owner_4555, Edge_Owner_4252, Edge_Owner_2122, Edge_Owner_3132, Edge_Owner_2425
Top 10 features importance:  2.24206, 1.64506, 1.33596, 1.27687, 1.00633, 0.92481, 0.85333, 0.81298, 0.76918, 0.75944


In [31]:
transition_df_prune = transition_df[transition_df.columns.drop(list(transition_df.filter(regex='Cell_Owner_')))]
transition_df_prune

,action,Edge_Owner_6061,Edge_Owner_6263,Edge_Owner_6465,Edge_Owner_5060,Edge_Owner_5262,Edge_Owner_5464,Edge_Owner_6171,Edge_Owner_6373,Edge_Owner_6575,...,Cell_Edge_Count_33,Cell_Edge_Count_34,Cell_Edge_Count_60,Cell_Edge_Count_61,Cell_Edge_Count_62,Cell_Edge_Count_63,Cell_Edge_Count_20,Cell_Edge_Count_64,Cell_Edge_Count_21,Cell_Edge_Count_22
0,"(0,3) -> (0,4)",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,"(1,0) -> (1,1)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(2,1) -> (2,2)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2,0
3,"(3,3) -> (4,3)",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"(3,5) -> (4,5)",0,0,0,0,0,0,0,0,0,...,1,2,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480728,"(2,0) -> (2,1)",0,0,0,0,0,0,-1,0,0,...,1,1,-1,-1,0,0,1,0,0,0
480729,"(1,0) -> (1,1)",0,0,0,0,0,0,-1,0,0,...,0,0,-1,-1,0,0,0,0,0,0
480730,"(3,1) -> (3,2)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
480731,"(6,5) -> (7,5)",0,0,0,0,0,0,0,0,2,...,0,0,-1,0,0,0,0,1,0,0


In [32]:
X = transition_df_prune.drop(columns=['action'])
y = transition_df_prune['action']

In [33]:
model = LogisticRegression(random_state=42, max_iter=500)
model.fit(X, y)
model.score(X, y)

0.48139819816821394

In [34]:
weights_importance(model, 0)

Action Name: (0,0) -> (0,1)
Top 10 features:  Cell_Edge_Count_00, Cell_Edge_Count_11, Edge_Owner_0102, Edge_Owner_0212, Edge_Owner_0001, Cell_Edge_Count_20, Edge_Owner_6373, Edge_Owner_3040, Edge_Owner_1323, Cell_Edge_Count_31
Top 10 features importance:  3.73925, 1.17159, 1.06355, 0.87115, 0.85123, 0.72005, 0.70149, 0.68300, 0.66256, 0.62711


In [35]:
weights_importance(model, 10)

Action Name: (0,5) -> (1,5)
Top 10 features:  Cell_Edge_Count_04, Edge_Owner_3334, Edge_Owner_5051, Edge_Owner_4041, Edge_Owner_1525, Edge_Owner_1222, Edge_Owner_6061, Edge_Owner_4151, Edge_Owner_2425, Edge_Owner_2434
Top 10 features importance:  5.01405, 1.49019, 1.38773, 1.31533, 1.29914, 1.23139, 1.23042, 1.18057, 1.17175, 1.16329


In [36]:
weights_importance(model, 50)

Action Name: (4,3) -> (4,4)
Top 10 features:  Edge_Owner_4344, Cell_Edge_Count_43, Edge_Owner_4445, Cell_Edge_Count_33, Edge_Owner_4243, Edge_Owner_4555, Edge_Owner_4252, Edge_Owner_2425, Edge_Owner_2122, Edge_Owner_3132
Top 10 features importance:  2.22815, 1.73441, 1.31303, 1.19053, 0.95021, 0.89211, 0.82438, 0.75515, 0.73172, 0.72990
